In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go
import pandas as pd
from urllib.request import urlopen
from datetime import datetime as dt
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     counties = json.load(response)

In [8]:
#counties

In [9]:
df = pd.read_csv('../../past_data/PM2.5_2019 (EPA).csv',sep = ',')
df_new = pd.read_csv('../counties.csv',sep = ',')
df_new['val'] = 0
df_new.dropna(subset=['FIPS #'],inplace = True)
#df = df.loc[:,['COUNTY_CODE','STATE','Site Name','SITE_LATITUDE','SITE_LONGITUDE','COUNTY','Daily Mean PM2.5 Concentration','CBSA_NAME']].groupby(['COUNTY_CODE','CBSA_NAME','STATE','Site Name','SITE_LATITUDE','SITE_LONGITUDE','COUNTY']).mean()
df = df.loc[:,['COUNTY_CODE','Daily Mean PM2.5 Concentration','COUNTY']].groupby(['COUNTY_CODE','COUNTY']).mean()
df.reset_index(inplace = True)
for index, row in df_new.iterrows():
    if(len(df[df['COUNTY'] == df_new.loc[index,' County Name '].strip()]) > 0):
        b = (df[df['COUNTY'] == df_new.loc[index,' County Name '].strip()])
        print(b.iloc[0]['Daily Mean PM2.5 Concentration'])
        df_new.loc[index,'val'] = b.iloc[0]['Daily Mean PM2.5 Concentration']
        print('hello')
    
df_new.reset_index(inplace=True)    
df_new['FIPS #'] = df_new['FIPS #']+48000
df_new['FIPS #'] = df_new['FIPS #'].astype(int).astype(str)
#48000 + (county_number * 2) - 1

8.39209621993127
hello
9.162187673514733
hello
9.36931818181818
hello
5.512464589235131
hello
9.564306784660754
hello
5.281818181818181
hello
9.103209459459451
hello
7.345303867403318
hello
7.637570621468922
hello
7.523037974683547
hello
8.570914826498438
hello
7.513898305084741
hello
8.933932657731296
hello
7.850123456790128
hello
10.600217864923737
hello
8.3970254957507
hello
6.667500000000002
hello
8.331338028169013
hello
6.404545454545459
hello
5.187292817679558
hello
7.385207100591716
hello
7.381593406593406
hello
6.649575070821529
hello
8.532834507042228
hello
8.242535211267606
hello
5.142647058823527
hello
8.745853658536586
hello
8.5703196347032
hello
10.69498607242339
hello


In [10]:
trace= go.Choroplethmapbox(geojson=counties,
                           locations=df_new['FIPS #'],
                           z=df_new['val'],
                           colorscale='ice',
                           colorbar_thickness=20,
                           text = df_new[' County Name '],
                           marker_line_color='white',customdata = df_new['FIPS #'], hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                            '<br> <b>Val </b>: %{z}<br>'+  '<extra></extra>'
                        )

In [11]:
fig= go.Figure(data = trace)
mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'                           
                            
fig.update_layout(title_text= 'Texas Heatmap',
                  title_x=0.5, width = 700, height=700,
                  mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                 accesstoken= mapboxt,style = 'basic',
                                 zoom=4.5,
                               ));
# fig.update_layout(hoverlabel=dict(
#         bgcolor="white", 
#         font_size=16, 
#         font_family="Rockwell"))
#fig.show()
df = df_new

In [ ]:

app = dash.Dash()
app.layout = html.Div([
    html.H1('Texas Air Quality Heatmap'),
    html.Div(id='text-content'),
    dcc.Graph(id='map', figure=fig),dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(1995, 8, 5),
        max_date_allowed=dt(2017, 9, 19),
        initial_visible_month=dt(2017, 8, 5),
        date=str(dt(2017, 8, 25, 23, 59, 59))
    )])

@app.callback(
    dash.dependencies.Output('text-content', 'children'),
    [dash.dependencies.Input('map', 'hoverData')])
def update_text(hoverData):
    if(hoverData == None):
        return "Hover over a county"
    s = df[df['FIPS #'] == hoverData['points'][0]['customdata']]
    # Blank county has blank /units/ of /parameter/
    return html.H3(
        '{} County has {} ppm of {}'.format(
            s.iloc[0][' County Name '],
            s.iloc[0]['val'],
            'PM2.5')
    )

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader = False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 056-796-787
Debugger PIN: 056-796-787
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
